[恐慌指數](https://money.cnn.com/data/fear-and-greed/)

巴菲特曾經說過：

> 在別人恐懼時我貪婪，在別人貪婪時我恐懼

這個現象真實地反映在股票市場裡，因為這不是一個單一指標，而是參考了市場中的：

- 市場波動率
- 大盤指數
- 市場追價的力道
- 避險的需求
- 把市場上重要的關鍵都納入，因此很容易判讀出目前市場的情緒與相對價格的高低

### 那麼什麼是恐懼與貪婪指數？

投資者受到兩種情緒的驅使：恐懼和貪婪。太多的恐懼會使股票跌至遠低於應有的水平。當投資者貪婪時，他們可能會以太高的價格出價。那麼，目前股票市場是什麼情緒？我們看看以下 7 個量化指標：

- 股票價格動能：標準普爾500指數（SPX）與125天移動平均線（MA）
- 股票價格走強：紐約證券交易所的股票數量達到52週的高點和低點
- 股票價格寬度：股票交易量的上升與下降之比。
- 看跌和看漲期權：看跌/看漲比率，將看漲看漲期權的交易量與看跌看跌期權的交易量進行比較
- 垃圾債券需求：投資級債券和垃圾債券的收益率之間的價差
- 市場波動率：衡量波動率的VIX
- 避風港需求：股票收益與國債收益之差

對於每個指標，我們查看它們相對於平均值的偏離量。我們以0-100的比例來衡量。指數越高，投資者的貪婪程度越高，50是中立的。 然後，我們將所有指標（均等加權）放在一起，以最終得到指數讀數。

## VIX 波動率指數 (Volatility Index)

也可以稱為 恐懼指數，可以理解為對 標普500 未來 30 天的隱含波動率預測，VIX 會每日更新，打個比方：

公司A和公司B的股票一樣有個價格，但它們的波動可能很不一樣。假如公司A的營運不容易有太大的意外發生，那麼它的股價會較穩定，不會有很大的波動發生。假如公司B的營運受很多變數影響，那麼它的股價波動會較大。

假如有口標普500在1000點的買權。假如未來市場走向是有50%的機會落在1050點，50%的機會落在950點。那麼這張買權有50%的機會值50塊，50%的機會不值錢(因為指數價950低於1000)。所以這張買權的價值便是25。

實際計算一下 VIX：

假如未來市場走向是各有25%的機會落在1100、1050、950和900點。波動範圍擴大。那麼這張買權有25%的機會賺100，25%的機會賺50，50%的機會不值錢。這時這張買權的價值增加為37.5。

通常 VIX 指數 超過 40 時，表示市場對未來的非理性恐慌，可能於短期內出現反彈。

相對的，當 VIX 指數低於 15，表示市場出現非理性繁榮，可能會伴隨著賣壓殺盤。 即使在1998年的金融風暴時，VIX指數也未曾超過60，VIX 指數不一定能準確預測走向，但是多少反映當時市場的氣氛

## 實作恐慌指數與貪婪指數

### 建立框架運行環境

In [ ]:
!pip install backtrader backtrader_plotting

In [ ]:
import backtrader as bt
import os
from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Blackly

In [ ]:
%matplotlib inline

### 引入策略

In [ ]:
from shutil import copyfile

copyfile(src = "../input/strategies/VIX.py", dst = "../working/VIX.py")

In [ ]:
from VIX import VIX

### 設定初始環境

In [ ]:
cerebro = bt.Cerebro()
cerebro.broker.setcash(1e5)

### 設定客製化資料格式 (標普 500 指數- 恐慌指數)

In [ ]:
class SPYVIXData(bt.feeds.GenericCSVData):
  lines = ('vixopen', 'vixhigh', 'vixlow', 'vixclose')

  params = (
    ('dtformat', '%Y-%m-%d'),
    ('date', 0),
    ('spyopen', 1),
    ('spyhigh', 2),
    ('spylow', 3),
    ('spyclose', 4),
    ('spyadjclose', 5),
    ('spyvolume', 6),
    ('vixopen', 7),
    ('vixhigh', 8),
    ('vixlow', 9),
    ('vixclose', 10)
  )


In [ ]:
class VIXData(bt.feeds.GenericCSVData):
  params = (
    ('dtformat', '%m/%d/%Y'),
    ('date', 0),
    ('vixopen', 1),
    ('vixhigh', 2),
    ('vixlow', 3),
    ('vixclose', 4),
    ('volume', -1),
    ('openinterest', -1)
  )


### 讀取資料

In [ ]:
csv_file = os.path.abspath('../input/quantitative-trading/SPY_VIX.csv')
vix_csv_file = os.path.abspath('../input/quantitative-trading/VIX.csv')

spyVixDataFeed = SPYVIXData(dataname=csv_file)
vixDataFeed = VIXData(dataname=vix_csv_file)
cerebro.adddata(spyVixDataFeed)
cerebro.adddata(vixDataFeed)

### 設定策略

In [ ]:
cerebro.addstrategy(VIX)

### 運行環境

In [ ]:
print('投資 > 起始資產 %.2f 💲' % cerebro.broker.getvalue())
cerebro.run()
print('投資 > 結束資產 %.2f 💲' % cerebro.broker.getvalue())

## 繪製報表

In [ ]:
b = Bokeh(style='bar', plot_mode='single', scheme=Blackly())
cerebro.plot(b)